In [30]:
import pandas as pd
import json
import os

In [31]:
target_user_id = 1

number_of_top_similarities = 2
number_of_newest_ratings = 5

In [32]:
# depth of this file in the project
file_depth = '../..'

user_similarities_file = f"rating_user_data/similarities_to_user_{target_user_id}.json"
save_sets_of_categories_file = 'rating_user_data/sets_of_categories_for_each_user.json'

with open(user_similarities_file, 'r') as f:
    similarities = json.load(f)

with open(save_sets_of_categories_file, 'r') as f:
    sets_of_categories = json.load(f)

In [33]:
similarities[0]['2']

0.5477225575051662

In [34]:
sets_of_categories[0]['2']

{'Animation': 0,
 'ForChildren': 0,
 'Comedy': 3,
 'Adventure': 0,
 'Fantasy': 0,
 'Romance': 1,
 'Drama': 2,
 'Action': 2,
 'Crime': 0,
 'Thriller': 2,
 'Horror': 0,
 'Sci-Fi': 1,
 'Documentary': 0,
 'War': 1,
 'Musical': 0,
 'Mystery': 0,
 'Film-Noir': 0,
 'Western': 0}

In [35]:
# load all the splits into a list
splits = []

for file in os.listdir('rating_user_data/splits/'):
    if file.endswith('.csv'):
        split = pd.read_csv('rating_user_data/splits/' + file)
        splits.append(split)

In [36]:
splits[0]

,user_id,movie_id,rating,timestamp,genres
0,1671,3342,5,974713449,Drama|War
1,2569,3929,4,973911951,Comedy
2,3747,1639,4,966138457,Drama|Romance
3,3821,3702,4,974759703,Action|Sci-Fi
4,5173,3147,5,961886251,Drama|Thriller
...,...,...,...,...,...
47995,1880,1028,4,975384642,ForChildren|Comedy|Musical
47996,2153,1219,5,974623040,Horror|Thriller
47997,5767,1580,4,958192067,Action|Adventure|Comedy|Sci-Fi
47998,4217,2458,4,965319593,Comedy|Crime


In [37]:
top_similarities = []

for similarity in similarities:
    sorted_similarities = sorted(similarity.items(), key=lambda item: item[1], reverse=True)
    top_similarities.append(sorted_similarities[:number_of_top_similarities])

In [38]:
top_similarities

[[('413', 0.9797958971132712), ('2831', 0.9689627902499088)],
 [('3619', 1), ('4540', 0.9707253433941511)],
 [('764', 0.9394421721142376), ('890', 0.9132483825423161)],
 [('1481', 0.9648363026488435), ('320', 0.9438798074485388)],
 [('2003', 1), ('5299', 1)],
 [('1092', 0.9566807697649699), ('4603', 0.938083151964686)],
 [('3560', 0.9432422182837987), ('2957', 0.9045340337332909)],
 [('993', 0.890870806374748), ('980', 0.8660254037844387)],
 [('333', 0.8944271909999159), ('885', 0.8888888888888888)],
 [('134', 0.8944271909999159), ('2379', 0.8944271909999159)],
 [('1359', 0.9486832980505138), ('2444', 0.9486832980505138)],
 [('707', 1), ('4772', 0.9669875568304563)]]

----------

In [39]:
def count_categories_occurrence(ratings):
    categories = {}
    for index, row in ratings.iterrows():
        for category in row['genres'].split('|'):
            if category in categories:
                categories[category] += 1
            else:
                categories[category] = 1
    return categories

In [40]:
reccomended_genres = []

# for each split except the last one (the last one is the test set)
for index, split in enumerate(splits[:-1]):

    # get the user_id of the user that is most similar to the target user
    # top_similarities[][][] -> [index of the split][index of the similarity][0 -> user_id, 1 -> similarity]
    user_x = int(top_similarities[index][0][0])

    # get the ratings of the target user and the user that is most similar to the target user
    # sort by timestamp in descending order
    # get the first 5 ratings
    target_user_ratings = split.loc[split['user_id'] == target_user_id].sort_values(by='timestamp', ascending=False)[:number_of_newest_ratings]
    user_x_ratings = split.loc[split['user_id'] == user_x].sort_values(by='timestamp', ascending=False)[:number_of_newest_ratings]

    categories_occurrence = count_categories_occurrence(target_user_ratings)

    max_value = max(categories_occurrence.values())
    keys_with_max_value = []
    for key, value in categories_occurrence.items():
        if value == max_value:
            keys_with_max_value.append(key)
    
    reccomended_genres.append(keys_with_max_value)

In [41]:
reccomended_genres

[['Animation', 'ForChildren', 'Comedy'],
 ['Fantasy'],
 ['ForChildren'],
 ['Drama'],
 ['Comedy', 'Crime', 'Drama', 'Thriller'],
 ['Animation', 'ForChildren', 'Action', 'Drama'],
 ['Animation', 'ForChildren'],
 ['Animation', 'ForChildren', 'Musical'],
 ['Musical'],
 ['ForChildren', 'Comedy', 'Musical', 'Drama'],
 ['Drama']]